In [65]:
# Read the input data as a multi-line string
input_data = """
041311
312202
3210
000000
000000
M.00..
000000
.....S
.S....
"""
N = 6
# Function to parse the input data
def parse_input(input_str):
    lines = input_str.strip().split('\n')
    row_constraints = [int(c) for c in lines[0]]
    col_constraints = [int(c) for c in lines[1]]
    fleet_counts = [int(c) for c in lines[2]]  # [Submarines, Destroyers, Cruisers, Battleships]
    grid = [list(line) for line in lines[3:]]
    return row_constraints, col_constraints, fleet_counts, grid

# Parse the input
row_constraints, col_constraints, fleet_counts, grid = parse_input(input_data)

# Display the parsed input
print("Row Constraints:", row_constraints)
print("Column Constraints:", col_constraints)
print("Fleet Counts (Submarines, Destroyers, Cruisers, Battleships):", fleet_counts)
print("Initial Grid:")
for row in grid:
    print(''.join(row))


Row Constraints: [0, 4, 1, 3, 1, 1]
Column Constraints: [3, 1, 2, 2, 0, 2]
Fleet Counts (Submarines, Destroyers, Cruisers, Battleships): [3, 2, 1, 0]
Initial Grid:
000000
000000
M.00..
000000
.....S
.S....


In [66]:
from collections import defaultdict

# Possible cell values
WATER = '.'
UNKNOWN = '0'

# Possible ship parts
SHIP_PARTS = ['S', '<', '>', '^', 'v', 'M']

# Initialize the assignment with pre-assigned cells
assignment = {}
for i in range(6):
    for j in range(6):
        cell = grid[i][j]
        if cell in SHIP_PARTS or cell == WATER:
            assignment[(i, j)] = cell

# Save the initial assignment to prevent modifying pre-filled cells
assignment_initial = assignment.copy()


# Variables: list of unassigned cell positions
variables = [(i, j) for i in range(6) for j in range(6) if (i, j) not in assignment]

# Domains: possible values for each variable
domains = {}

# Domains: possible values for each unassigned variable
domains = {}
for var in variables:
    domains[var] = [WATER] + SHIP_PARTS


In [67]:
def is_ship_part(value):
    return value in SHIP_PARTS


def is_water(value):
    return value == WATER


In [68]:
def get_neighbors(i, j):
    neighbors = []
    for di in [-1, 0, 1]:
        for dj in [-1, 0, 1]:
            ni, nj = i + di, j + dj
            if 0 <= ni < 6 and 0 <= nj < 6 and (di != 0 or dj != 0):
                neighbors.append((ni, nj))
    return neighbors


In [69]:
def get_adjacent_cells(i, j):
    adjacents = []
    for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        ni, nj = i + di, j + dj
        if 0 <= ni < 6 and 0 <= nj < 6:
            adjacents.append((ni, nj))
    return adjacents


In [70]:
def ship_shape_is_valid(assignment, var):
    i, j = var
    value = assignment[var]

    if value == 'S':
        # Submarine must be alone
        for ni, nj in get_adjacent_cells(i, j):
            neighbor_value = assignment.get((ni, nj), None)
            if neighbor_value and is_ship_part(neighbor_value):
                return False
        return True

    elif value in ['<', '>', '^', 'v', 'M']:
        # For other ship parts, check if they form valid ships
        # We can perform this check after the full assignment is done
        # For partial assignments, we may not have enough information
        return True

    return True  # For water or unassigned


In [71]:
def row_col_constraints_satisfied(assignment):
    # Check row constraints
    for i in range(6):
        count = sum(1 for j in range(6) if is_ship_part(assignment.get((i, j), '0')))
        if count > row_constraints[i]:
            return False

    # Check column constraints
    for j in range(6):
        count = sum(1 for i in range(6) if is_ship_part(assignment.get((i, j), '0')))
        if count > col_constraints[j]:
            return False

    return True


In [72]:
def explore_ship(assignment, i, j, visited):
    stack = [(i, j)]
    ship_cells = []
    while stack:
        ci, cj = stack.pop()
        if (ci, cj) in visited:
            continue
        visited.add((ci, cj))
        value = assignment.get((ci, cj), WATER)
        if is_ship_part(value):
            ship_cells.append((ci, cj))
            # Add adjacent cells (orthogonal only) to the stack
            for ni, nj in get_adjacent_cells(ci, cj):
                neighbor_value = assignment.get((ni, nj), WATER)
                if is_ship_part(neighbor_value):
                    stack.append((ni, nj))
    # Print the ship cells explored
    
    return ship_cells


In [73]:
def is_ship_complete(assignment, ship_cells):
    for i, j in ship_cells:
        # Vérifier si toutes les cellules adjacentes dans la direction du navire sont assignées
        neighbors = get_adjacent_cells(i, j)
        for ni, nj in neighbors:
            if (ni, nj) in assignment:
                continue
            else:
                # Si une cellule voisine n'est pas assignée, le navire n'est pas encore complet
                return False
    return True


In [74]:
def ship_size_to_type(ship_length):
    if ship_length == 1:
        return 'S'  # Sous-marin
    elif ship_length == 2:
        return 'D'  # Destroyer
    elif ship_length == 3:
        return 'C'  # Croiseur
    elif ship_length == 4:
        return 'B'  # Cuirassé
    else:
        return None  # Taille de navire invalide


In [75]:
def fleet_constraints_satisfied(assignment, complete=False):
    fleet_counts_current = {'S': 0, 'D': 0, 'C': 0, 'B': 0}  # Sous-marins, Destroyers, Croiseurs, Cuirassés

    visited = set()

    for i in range(N):
        for j in range(N):
            if (i, j) not in visited and is_ship_part(assignment.get((i, j), WATER)):
                ship_cells = explore_ship(assignment, i, j, visited)
                ship_length = len(ship_cells)

                # Si le navire est incomplet, ne pas le compter pour le moment
                if not is_ship_complete(assignment, ship_cells):
                    continue

                ship_type = ship_size_to_type(ship_length)
                if ship_type:
                    fleet_counts_current[ship_type] += 1
                else:
                    # Navire de taille invalide
                    return False

                # Vérifier si nous avons dépassé les contraintes de flotte
                required_counts = {
                    'S': fleet_counts[0],
                    'D': fleet_counts[1],
                    'C': fleet_counts[2],
                    'B': fleet_counts[3],
                }
                for ship_type, count in fleet_counts_current.items():
                    if count > required_counts[ship_type]:
                        return False

    if complete:
        # Vérifier si nous avons exactement le nombre requis de navires de chaque type
        required_counts = {
            'S': fleet_counts[0],
            'D': fleet_counts[1],
            'C': fleet_counts[2],
            'B': fleet_counts[3],
        }
        for ship_type in fleet_counts_current:
            if fleet_counts_current[ship_type] != required_counts[ship_type]:
                return False

    return True


In [76]:
def are_parts_of_same_ship(cell1, cell2, assignment):
    # Use depth-first search (DFS) or breadth-first search (BFS) to find all connected ship parts
    # starting from cell1 and see if cell2 is among them.

    visited = set()
    stack = [cell1]

    while stack:
        current_cell = stack.pop()
        if current_cell == cell2:
            return True  # Both cells are part of the same ship
        if current_cell in visited:
            continue
        visited.add(current_cell)
        i, j = current_cell
        value = assignment.get((i, j), None)
        if is_ship_part(value):
            # Add adjacent cells (orthogonal) that are ship parts
            for ni, nj in get_adjacent_cells(i, j):
                neighbor_value = assignment.get((ni, nj), None)
                if neighbor_value and is_ship_part(neighbor_value):
                    stack.append((ni, nj))
    return False  # Cells are not part of the same ship


In [77]:
def get_orthogonal_neighbors(i, j):
    orthogonals = []
    for di, dj in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        ni, nj = i + di, j + dj
        if 0 <= ni < 5 and 0 <= nj < 5:
            orthogonals.append((ni, nj))
    return orthogonals


In [78]:
def get_diagonal_neighbors(i, j):
    diagonals = []
    for di, dj in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
        ni, nj = i + di, j + dj
        if 0 <= ni < 5 and 0 <= nj < 5:
            diagonals.append((ni, nj))
    return diagonals


In [79]:
def is_consistent(var, value, assignment):
    i, j = var
    
    # Do not modify pre-filled cells
    if (i, j) in assignment_initial and assignment_initial[(i, j)] != value:
        return False

    # Create a temporary assignment including the new value
    temp_assignment = assignment.copy()
    temp_assignment[var] = value

    if is_ship_part(value):
        # Check diagonal neighbors
        for ni, nj in get_diagonal_neighbors(i, j):
            neighbor_value = temp_assignment.get((ni, nj), None)
            if is_ship_part(value) and neighbor_value and is_ship_part(neighbor_value):
                # Ships are touching diagonally - violation
                
                return False

        # Constraint for orthogonal neighbors
        for ni, nj in get_orthogonal_neighbors(i, j):
            neighbor_value = temp_assignment.get((ni, nj), None)
            if is_ship_part(value) and neighbor_value and is_ship_part(neighbor_value):
                if not are_parts_of_same_ship(var, (ni, nj), temp_assignment):
                # Ships are touching orthogonally - violation if not same ship
                    
                    return False


    # Constraint 2: Ship shapes must be valid
    # Check if the ship parts are connected correctly
    if not ship_shape_is_valid(temp_assignment, var):
        
        return False

    # Constraint 3: Row and Column constraints
    if not row_col_constraints_satisfied(temp_assignment):
        
        return False

    # Constraint 4: Fleet constraints
    if not fleet_constraints_satisfied(temp_assignment, complete=False):
        
        return False

    return True


In [80]:
def select_unassigned_variable(assignment):
    # Minimum Remaining Value (MRV) heuristic
    unassigned_vars = [v for v in variables if v not in assignment]
    # Return the variable with the smallest domain
    return min(unassigned_vars, key=lambda var: len(domains[var]))

def order_domain_values(var, assignment):
    # Least Constraining Value (LCV) heuristic
    return domains[var]


In [81]:
def check_ship_ends(assignment, ship_cells):
    ship_cells.sort()
    ship_length = len(ship_cells)
    if ship_length == 1:
        # Should be 'S'
        i, j = ship_cells[0]
        if assignment[(i, j)] != 'S':
            
            return False
    else:
        # Check the ends and middle parts
        if all(cj == ship_cells[0][1] for ci, cj in ship_cells):
            # Vertical ship
            ship_cells.sort(key=lambda x: x[0])  # Sort by row
            expected_parts = ['^'] + ['M'] * (ship_length - 2) + ['v']
        elif all(ci == ship_cells[0][0] for ci, cj in ship_cells):
            # Horizontal ship
            ship_cells.sort(key=lambda x: x[1])  # Sort by column
            expected_parts = ['<'] + ['M'] * (ship_length - 2) + ['>']
        else:
            
            return False

        for idx, (i, j) in enumerate(ship_cells):
            if assignment[(i, j)] != expected_parts[idx]:
                
                return False
    return True


In [82]:
def validate_ship_shapes(assignment):
    visited = set()
    for i in range(6):
        for j in range(6):
            if (i, j) not in visited and is_ship_part(assignment.get((i, j), WATER)):
                ship_cells = explore_ship(assignment, i, j, visited)
                ship_length = len(ship_cells)

                # Determine orientation
                if ship_length > 1:
                    is_horizontal = all(ci == ship_cells[0][0] for ci, cj in ship_cells)
                    is_vertical = all(cj == ship_cells[0][1] for ci, cj in ship_cells)
                    if not (is_horizontal or is_vertical):
                        
                        return False

                # Check ship ends
                if not check_ship_ends(assignment, ship_cells):
                    
                    return False
    return True


In [83]:
def final_assignment_is_valid(assignment):
    # Check row constraints
    for i in range(6):
        count = sum(1 for j in range(6) if is_ship_part(assignment.get((i, j), WATER)))
        if count != row_constraints[i]:
            
            return False

    # Check column constraints
    for j in range(6):
        count = sum(1 for i in range(6) if is_ship_part(assignment.get((i, j), WATER)))
        if count != col_constraints[j]:
            
            return False

    # Check fleet constraints
    if not fleet_constraints_satisfied(assignment, complete=True):
        
        return False

    # Validate ship shapes and placements
    if not validate_ship_shapes(assignment):
        
        return False

    return True


In [84]:
# Global counter for recursive calls
recursive_calls = 0

In [85]:
def backtrack(assignment):
    global recursive_calls
    recursive_calls += 1

    if len(assignment) == 36:  # Total number of cells in a 6x6 grid
        if final_assignment_is_valid(assignment):
            return assignment
        else:
            
            return None

    var = select_unassigned_variable(assignment)
    if var is None:
        return None

    for value in order_domain_values(var, assignment):
        if is_consistent(var, value, assignment):
            assignment[var] = value
            
            result = backtrack(assignment)
            if result is not None:
                return result
            del assignment[var]
            
               
    return None


In [86]:
def solve_puzzle():

                
    

    result = backtrack(assignment.copy())
    if result is not None:
        # Construct the solved grid
        solved_grid = [['' for _ in range(6)] for _ in range(6)]
        for (i, j), value in result.items():
            solved_grid[i][j] = value
        # Replace any remaining unassigned cells with water
        for i in range(6):
            for j in range(6):
                if solved_grid[i][j] == '':
                    solved_grid[i][j] = WATER
        return solved_grid
    else:
        print("No solution found.")
        return None

# Solve the puzzle
solved_grid = solve_puzzle()

# Output the solution
if solved_grid:
    print("Solved Grid:")
    for row in solved_grid:
        print(''.join(row))


Solved Grid:
......
^.<>.S
M.....
v.<>..
.....S
.S....


### **FORWARD**


In [87]:
def forward_check(var, value, assignment, domains):
    i, j = var
    temp_assignment = assignment.copy()
    temp_assignment[var] = value

    # Obtenir les voisins non assignés
    neighbors = get_adjacent_cells(i, j) + get_diagonal_neighbors(i, j)
    for neighbor in neighbors:
        if neighbor in variables and neighbor not in assignment:
            # Mettre à jour le domaine du voisin
            new_domain = []
            for neighbor_value in domains[neighbor]:
                if is_consistent(neighbor, neighbor_value, temp_assignment):
                    new_domain.append(neighbor_value)
            if not new_domain:
                # Domaine vide, inconsistance détectée
                return False
            else:
                domains[neighbor] = new_domain
    return True


In [88]:
import copy

In [89]:
def Forward_backtrack(assignment, domains):
    global recursive_calls
    recursive_calls += 1

    if len(assignment) == 36:
        if final_assignment_is_valid(assignment):
            return assignment
        else:
            return None

    var = select_unassigned_variable(assignment)
    if var is None:
        return None

    for value in order_domain_values(var, assignment):
        if is_consistent(var, value, assignment):
            assignment[var] = value
            # Faire une copie profonde des domaines
            new_domains = copy.deepcopy(domains)
            # Appliquer le Forward Checking
            if forward_check(var, value, assignment, new_domains):
                result = Forward_backtrack(assignment, new_domains)
                if result is not None:
                    return result
            # Backtracking
            del assignment[var]
    return None


In [90]:
def solve_puzzle_forward():

                
    

    result = Forward_backtrack(assignment.copy(),domains)
    if result is not None:
        # Construct the solved grid
        solved_grid = [['' for _ in range(6)] for _ in range(6)]
        for (i, j), value in result.items():
            solved_grid[i][j] = value
        # Replace any remaining unassigned cells with water
        for i in range(6):
            for j in range(6):
                if solved_grid[i][j] == '':
                    solved_grid[i][j] = WATER
        return solved_grid
    else:
        print("No solution found.")
        return None

# Solve the puzzle
solved_grid = solve_puzzle_forward()

# Output the solution
if solved_grid:
    print("Solved Grid:")
    for row in solved_grid:
        print(''.join(row))


Solved Grid:
......
^.<>.S
M.....
v.<>..
.....S
.S....


### **AC3**

In [91]:
# Fonction pour vérifier si deux valeurs sont compatibles
def is_consistent_with(Xi, x, Xj, y):
    temp_assignment = {Xi: x, Xj: y}
    return is_consistent(Xi, x, temp_assignment) and is_consistent(Xj, y, temp_assignment)

# Fonction pour réviser le domaine de Xi
def revise(domains, Xi, Xj):
    revised = False
    to_remove = []
    for x in domains[Xi]:
        # Vérifier s'il existe une valeur y dans domains[Xj] telle que (x, y) est cohérent
        if not any(is_consistent_with(Xi, x, Xj, y) for y in domains[Xj]):
            to_remove.append(x)
            revised = True
    for x in to_remove:
        domains[Xi].remove(x)
    return revised

# Algorithme AC-3
def ac3(domains):
    queue = []
    for Xi in variables:
        neighbors = get_adjacent_cells(Xi[0], Xi[1]) + get_diagonal_neighbors(Xi[0], Xi[1])
        for Xj in neighbors:
            if Xj in variables:
                queue.append((Xi, Xj))
    while queue:
        (Xi, Xj) = queue.pop(0)
        if revise(domains, Xi, Xj):
            if not domains[Xi]:
                return False  # Domaine vidé, incohérence détectée
            neighbors = get_adjacent_cells(Xi[0], Xi[1]) + get_diagonal_neighbors(Xi[0], Xi[1])
            for Xk in neighbors:
                if Xk != Xj and Xk in variables:
                    queue.append((Xk, Xi))
    return True


In [92]:
def backtrack_ac3(assignment, domains):
    global recursive_calls
    recursive_calls += 1

    if len(assignment) == N * N:
        if final_assignment_is_valid(assignment):
            return assignment
        else:
            return None

    var = select_unassigned_variable(assignment)
    if var is None:
        return None

    for value in order_domain_values(var, assignment):
        if value not in domains[var]:
            continue
        if is_consistent(var, value, assignment):
            assignment[var] = value
            # Faire une copie profonde des domaines
            new_domains = copy.deepcopy(domains)
            # Assigner la valeur à var dans les nouveaux domaines
            new_domains[var] = [value]
            # Appliquer AC-3
            if ac3(new_domains):
                result = backtrack_ac3(assignment, new_domains)
                if result is not None:
                    return result
            # Backtracking
            del assignment[var]
    return None


In [93]:
def solve_puzzle_ac3():
    # Initialiser les domaines
    domains_ac3 = {}
    for var in variables:
        domains_ac3[var] = [WATER] + SHIP_PARTS

    # Appliquer AC-3 avant de commencer le backtracking
    if not ac3(domains_ac3):
        print("AC-3 a détecté une incohérence avant de commencer.")
        return None

    assignment_ac3 = assignment.copy()
    result = backtrack_ac3(assignment_ac3, domains_ac3)
    if result is not None:
        # Construire la grille résolue
        solved_grid = [['' for _ in range(N)] for _ in range(N)]
        for (i, j), value in result.items():
            solved_grid[i][j] = value
        # Remplir les cellules restantes depuis l'assignation initiale
        for i in range(N):
            for j in range(N):
                if solved_grid[i][j] == '':
                    solved_grid[i][j] = grid[i][j]
        return solved_grid
    else:
        print("Aucune solution trouvée avec AC-3.")
        return None


In [94]:
# Réinitialiser le compteur d'appels récursifs
recursive_calls = 0

# Résoudre le puzzle en utilisant AC-3
solved_grid_ac3 = solve_puzzle_ac3()

# Afficher la solution
if solved_grid_ac3:
    print("Grille résolue avec AC-3 :")
    for row in solved_grid_ac3:
        print(''.join(row))
    print(f"Appels récursifs : {recursive_calls}")
else:
    print("Aucune solution trouvée avec AC-3.")


Grille résolue avec AC-3 :
......
^.<>.S
M.....
v.<>..
.....S
.S....
Appels récursifs : 482185
